#### Criando banco de dados

In [ ]:
import sqlite3

conn = sqlite3.connect('stackoverflow.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS questions(
    id INTERGER PRIMARY KEY,
    title TEXT,
    tag TEXT,
    body TEXT 
)
''')

conn.commit()
print('Banco de dados criado com sucesso')

#### Função que adiciona os dados a base

In [25]:
def save_question(questions,tag):
    for question in questions:
        cursor.execute(''' INSERT OR REPLACE INTO questions (id,title,tag,body) VALUES (?,?,?,?) ''',
            (question['question_id'],question['title'],tag,question['body']))
    conn.commit()
    print(f'Tema {question} salvo na base')


#### Caputra as perguntas do stackoverflow

In [ ]:
import httpx


url = "https://api.stackexchange.com/2.3/questions"
site = "stackoverflow"
api_key = "rl_sVA4bTddeGTtj2EFpzt2RKu7r"
page_size = 100
max_page = 5
assuntos = ['pandas','tensorflow','machine-learning','dataframe','scikit-learn',
            'data-visualization']

def get_stackoverflow_questions(tag, page):
    params = {
        "order": "desc",
        "sort": "activity",
        "tagged": tag,
        "site": site,
        "page": page,
        "filter" : "withbody",
        "page_size" : page_size,
        "key" : api_key,
        "is_answered" : 'True'
    }
    response = httpx.get(url, params=params)
    response.raise_for_status()  # Levanta um erro se a requisição falhar
    return response.json()

# Exemplo de uso

base = {
    'pandas' : [],
    'tensorflow': [],
    'machine-learning': [],
    'dataframe' : [],
    'scikit-learn' : [],
    'data-visualization' : []
}

for assunto in assuntos:
  for i in range(1,max_page+1):
      question = get_stackoverflow_questions(assunto, i)
      for item in question['items']:
        body ={
          'question_id' : item['question_id'],
          'title' : item['title'],
          'body' :  item['body']
        }
        base[assunto].append(body)


for tema in base:
    save_question(base[tema],tema)

#### Trata o corpo do texto recebidos pela API

In [85]:
from bs4 import BeautifulSoup


def pross_text(text):
    soup = BeautifulSoup(text,'html.parser')

    #Queremos que a pergunta saia na mesma ordem mas com a distinção de texto e codigo 

    elements_order = [] #Criamos uma lista para amarzenar o texto na ordem do texto original

    #Itera pelos filhos direto do corpo do texto
    for elem in soup.find_all(['p','pre']):
        if elem.name == 'p':
            text = elem.get_text().strip()
            if text:
                elements_order.append(('texto',text))
        elif elem.name == 'pre':
            code = elem.get_text().strip()
            if code:
                elements_order.append(('codigo',code))

    texto_final = ''

    for tipo,conteudo in elements_order:
        if tipo == 'texto':
            texto_final += conteudo + '\n'
        elif tipo == 'codigo':
            texto_final += '```python \n' + conteudo + '\n'

    
    return texto_final

#### Atualiza a base de dados com os texto tratados

In [ ]:
cursor.execute('Select id,body from questions')
text = cursor.fetchall()

sql_update = "UPDATE questions SET body = ? WHERE id = ?"
for question in text:
    text = pross_text(question[1])
    cursor.execute(sql_update,(text,question[0]))
    conn.commit()

print('Tabela atualizada')
    